# Remove Tissue Pvalues

Remove tissues one at a time and compare p-values

# Inputs

In [3]:
import rnaseq_lib3 as r
import pandas as pd
import scipy.stats as st
import pymc3 as pm
import numpy as np
import time
import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
pm.Normal?

In [3]:
# Read in centered data
df = pd.read_hdf('/mnt/data/expression/tcga_gtex_tpm_norm_filt.hd5')
# Subset
gtex = df[df.label == 'gtex'].sort_values('tissue')
normal = df[df.label == 'tcga-normal'].sort_values('tissue')
tumor = df[df.label == 'tcga-tumor'].sort_values('tissue')
# Read in drug genes
genes = df.columns[5:]
drug_genes = [x.split('\t')[0] for x in open('../../data/druggable-genes.tsv', 'r').readlines()]
drug_genes = [x for x in drug_genes if x in df.columns]

# Model

## Model Spec

In [1]:
sample = tumor.loc['TCGA-FY-A3I4-01']
print(f"Sample comes from tissue: {sample.tissue}")
# We'll pick a subset of normal tissue for model training
test = normal
training_genes = r.outlier.select_k_best_genes(test, genes, 'tissue', 50)
classes = sorted(test.tissue.unique())
ncats = len(classes)
print(f'{ncats} background datasets and {len(training_genes)} genes')

NameError: name 'tumor' is not defined

In [ ]:
ys = {}
for gene in training_genes:
    for dataset in classes:
        name = f'{gene}-{dataset}'
        ys[name] = st.norm.fit(test[test.tissue == dataset][gene])

## Run Model

In [24]:
out_dir = '/mnt/research_serializations/Obs-Stoch-Ys-Remove-Tissue-Pvalues'
out_path = os.path.join(out_dir, f'{sample.id}-logit.pkl')
if os.path.exists(out_path):
    logit_model = pickle.load(open(out_path, 'rb'))
    trace_logit, model_logit = logit_model['trace'], logit_model['model']
else:
    t0 = time.time()
    with model_logit:
        trace_logit = pm.sample(target_accept=0.9)
    runtime = (time.time() - t0) / 60
    # Save
    logit_model = {}
    logit_model['trace'], logit_model['model'] = trace, model
    with open(out_path, 'wb') as f:
        pickle.dump(logit_model, f)